# CrewAI and Weaviate Getting Started Example

## Import Libraries and Dependencies

At the time of creating this notebook, we're using the `weaviate-client` version `4.15.4` and `crewai` version `0.134.0`.

In [ ]:
!pip install --quiet weaviate-client==4.15.4

In [ ]:
!pip install --quiet crewai==0.134.0

In [1]:
import os
from dotenv import load_dotenv

from crewai_tools import WeaviateVectorSearchTool
import weaviate
from weaviate.classes.init import Auth

from crewai import Agent
from crewai_tools import WeaviateVectorSearchTool
from crewai import Task
from crewai import Crew, Process

load_dotenv()


True

## Connect to Weaviate

In [6]:
WCD_CLUSTER_URL = os.getenv("WCD_CLUSTER_URL")
WCD_CLUSTER_KEY = os.getenv("WCD_CLUSTER_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WCD_CLUSTER_URL,
    auth_credentials=Auth.api_key(WCD_CLUSTER_KEY),
    headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")}
)

print(client.is_ready())

True


## Connect to Existing Weaviate Collection

You'll want to connect to your existing Weaviate collection. If you don't have one already, you can follow this import notebook [here](/integrations/Weaviate-Import-Example.ipynb)!

In [7]:
weaviate_collection = client.collections.get("WeaviateBlogChunk")

## Initialize the Weaviate Retriever Tool

In [8]:
weaviate_tool = WeaviateVectorSearchTool(
    collection_name='WeaviateBlogChunk', 
    limit=4,
    weaviate_cluster_url=WCD_CLUSTER_URL,
    weaviate_api_key=WCD_CLUSTER_KEY,
)

## Build the Agent

In [15]:
research_agent = Agent(
    role="Write specialized features based on the customers use case",
    goal="Inform the user on specific Weaviate features",
    backstory="You are a helpful assistant that can answer questions with the help of the WeaviateVectorSearchTool.",
    llm="gpt-4o-mini",
    tools=[weaviate_tool],
)

In [16]:
agent_task = Task(
    description="""
    The customer is a financial institution that is interested in using Weaviate as their vector database. 
    Research a few of the latest features from Weaviate and then suggest a few that are relevant to the customer.
    """,
    expected_output="A one paragraph summary on why the customer should implement the latest Weaviate features.",
    agent=research_agent
)

In [17]:
blog_crew = Crew(
    agents=[research_agent],
    tasks=[agent_task],
    process=Process.sequential,
    verbose=True
)

## Query Time

In [18]:
result = blog_crew.kickoff()

print(result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e72e5972-7861-4da1-a112-4fbbf631a8c8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Write specialized features based on the customers use case                                              │
│                                                                                                                 │
│  Task:                                                                                                          │
│      The customer is a financial institution that is interested in using Weaviate as their vector database.     │
│      Research a few of the latest features from Weaviate and then suggest a few that are relevant to the        │
│  customer.                                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x146756580>


╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Write specialized features based on the customers use case                                              │
│                                                                                                                 │
│  Thought: I should first research the latest features from Weaviate that would be relevant to a financial       │
│  institution.                                                                                                   │
│                                                                                                                 │
│  Using Tool: WeaviateVectorSearchTool                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x1465ebb60>


╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Write specialized features based on the customers use case                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Implementing the latest features of Weaviate can greatly enhance the operations of financial institutions by   │
│  introducing capabilities such as native multi-tenancy support, which allows different user groups to securely  │
│  access the same application while maintaining strict compliance with regulations like GDPR. This is critical   │
│  in the financial sector, where data privacy is paramount. Furthermore, the recent introduction of features     │
│  like AutoPQ (automatic product quantization) and flat indexing enable faster query responses and efficient     │
│  handling of vast amounts of financial data. Weaviate's support for various embedding providers ensures that    │
│  institutions can seamlessly integrate advanced machine learning and AI technologies into their workflows,      │
│  thereby optimizing their data analysis and retrieval processes. With these updates, Weaviate not only          │
│  supports the specific requirements of the finance industry but also provides a secure, scalable, and           │
│  efficient platform for managing complex datasets.                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0d4b3335-aefb-4048-9a0c-763611ab7997                                                                     │
│  Agent: Write specialized features based on the customers use case                                              │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e72e5972-7861-4da1-a112-4fbbf631a8c8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Implementing the latest features of Weaviate can greatly enhance the operations of financial     │
│  institutions by introducing capabilities such as native multi-tenancy support, which allows different user     │
│  groups to securely access the same application while maintaining strict compliance with regulations like       │
│  GDPR. This is critical in the financial sector, where data privacy is paramount. Furthermore, the recent       │
│  introduction of features like AutoPQ (automatic product quantization) and flat indexing enable faster query    │
│  responses and efficient handling of vast amounts of financial data. Weaviate's support for various embedding   │
│  providers ensures that institutions can seamlessly integrate advanced machine learning and AI technologies     │
│  into their workflows, thereby optimizing their data analysis and retrieval processes. With these updates,      │
│  Weaviate not only supports the specific requirements of the finance industry but also provides a secure,       │
│  scalable, and efficient platform for managing complex datasets.                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Implementing the latest features of Weaviate can greatly enhance the operations of financial institutions by introducing capabilities such as native multi-tenancy support, which allows different user groups to securely access the same application while maintaining strict compliance with regulations like GDPR. This is critical in the financial sector, where data privacy is paramount. Furthermore, the recent introduction of features like AutoPQ (automatic product quantization) and flat indexing enable faster query responses and efficient handling of vast amounts of financial data. Weaviate's support for various embedding providers ensures that institutions can seamlessly integrate advanced machine learning and AI technologies into their workflows, thereby optimizing their data analysis and retrieval processes. With these updates, Weaviate not only supports the specific requirements of the finance industry but also provides a secure, scalable, and efficient platform for managing complex d